In [55]:
import subprocess
import io
import fetch_data as fd
import numpy as np


categories = ['earn','crude']
numberOfTraining = [10,10]
numberOfTesting = [7,2]


trainData,trainLabels, testData,testLabel = fd.loadData(categories,numberOfTraining,numberOfTesting)
# from subprocess import Popen, PIPE, STDOUT

# Compile the fast_recursive_kernel
test = subprocess.Popen(["g++","fast_recursive_kernel.cpp","-o","fast_recursive_kernel.out"], stdout=subprocess.PIPE)
output_compile = test.communicate()[0]
# print(output_compile.decode('ascii'))


def parseMatrix(matrix):
    num_rows = int(matrix[0])
    num_cols = int(matrix[1])
    K = np.zeros([num_rows,num_cols])
    counter = 2
    for i in range(num_rows):
        for j in range(num_cols):
            K[i,j] = float(matrix[counter])
            counter = counter + 1
    return K

In [67]:
# Test fast_recursive_kernel
n = 2
l = 0.5
s = trainData
# t = trainData + trainData
t = testData
# def marcus_recursive_kernel(s,t,n,l):
the_strings = b''
the_args = bytes(str(len(s)),'ascii')+ b' ' + bytes(str(len(t)),'ascii')+ b' '
for i in s:
    the_strings = the_strings + bytes(i,'ascii')
    the_args = the_args + bytes(str(len(i)),'ascii') + b' '
for i in t:
    the_strings = the_strings + bytes(i,'ascii')
    the_args = the_args + bytes(str(len(i)),'ascii') + b' '
the_args = the_args + bytes(str(n), 'ascii')+ b' ' + bytes(str(l), 'ascii')
fast_kernel = subprocess.Popen(["./fast_recursive_kernel.out"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
output_test = fast_kernel.communicate(input=b'1 '+the_args+the_strings)[0]
print(output_test.decode('utf-8'), flush=True)

output_test_list = output_test.decode('utf-8').split()
#     K = parseMatrix(output_test_list)
#     return K

# K = marcus_recursive_kernel(trainData,testData,n,l)


2871
HHHHHEEEELLLLOOOOO

0 143 20
0 143 20
0 143 20
0 143 20
1 2871 20
1 2871 20
1 2871 20
1 2871 20
2 123 20
2 123 20
2 123 20
2 123 20
3 849 20
3 849 20
3 849 20
3 849 20
4 837 20
4 837 20
4 837 20
4 837 20
5 89 20
5 89 20
5 89 20
5 89 20
6 104 20
6 104 20
6 104 20
6 104 20
7 925 20
7 925 20
7 925 20
7 925 20
8 201 20
8 201 20
8 201 20
8 201 20
9 440 20
9 440 20
9 440 20
9 440 20
10 1459 20
10 1459 20
10 1459 20
10 1459 20
11 368 20
11 368 20
11 368 20
11 368 20
12 352 20
12 352 20
12 352 20
12 352 20
13 310 20
13 310 20
13 310 20
13 310 20
14 174 20
14 174 20
14 174 20
14 174 20
15 448 20
15 448 20
15 448 20
15 448 20
16 1089 20
16 1089 20
16 1089 20
16 1089 20
17 1161 20
17 1161 20
17 1161 20
17 1161 20
18 146 20
18 146 20
18 146 20
18 146 20
19 508 20
19 508 20
19 508 20
19 508 20
HHHHHEEEELLLLOOOOO

20 9 0.7730406729655087 0.7065003044281508 0.7709364950459572 0.9590795029369392 0.7251109795076011 0.7441680218985288 0.5255587303535245 0.07373937008352371 0.5410510033566484 0.5685

In [41]:
[len(i) for i in testData]

[263, 236, 130, 299, 92, 147, 148, 919, 472]

In [66]:
len(trainData[1])

2871

In [10]:
%timeit marcus_recursive_kernel(trainData,trainData,5,l)

7.6 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
marcus_recursive_kernel(trainData,testData,5,l)

IndexError: list index out of range

In [12]:
testData

['pall corp lt pll sets quarterly dividend qtly div cts vs cts prior pay may one record april ',
 'rjr nabisco first quarter net include mln dlr gain mln dlr charge rjr nabisco first quarter net include mln dlr gain mln dlr charge ',
 'cleopatra kohlique inc lt cleo year net year ended march shr three cts vs cts net vs revs mln vs mln ',
 'fleet financial group st qtr shr cts vs cts fleet financial group st qtr shr cts vs cts ',
 'global natural resources inc lt gnr th qtr loss oper shr loss five cts vs loss nil ope net loss vs loss revs vs mln avg shrs mln vs mln year oper shr profit cts vs loss one ct oper net profit vs loss revs mln vs mln avg shrs mln vs mln note net excludes extraordinary tax charges dlrs vs dlrs quarter credits dlrs vs dlrs year ',
 'qms inc lt aqm th qtr oct net shr cts vs cts net vs revs mln vs mln avg shrs vs year shr cts vs cts net vs revs mln vs mln avg shrs vs ',
 'hartmarx corp lt hmx nd qtr may net shr cts vs cts net vs sales mln vs mln six mths shr cts v